In [ ]:
# default_exp inference

In [ ]:
#export
import tensorflow as tf
from tensorflow.keras.models import load_model
from pathlib import Path
from PIL import Image
import os
import cv2
import numpy as np
from tqdm.notebook import tqdm

from xml.etree.ElementTree import Element, SubElement, tostring 
import pprint
from xml.dom.minidom import parseString

from two_stage_detector.bbox import *
from two_stage_detector.inference import *

from fastcore.test import *

In [ ]:
from matplotlib import pyplot as plt
from IPython.display import clear_output

In [ ]:
%config Completer.use_jedi = False

In [ ]:
path_det1 = '/home/addfor/projects/smart_tray/20_5006v02-intp-detection-foodv01/det1_model/saved_model'
path_det2 = '/home/addfor/projects/smart_tray/20_5006v02-intp-detection-foodv01/det2_model/saved_model'
det1 = tf.saved_model.load(path_det1)
det2 = tf.saved_model.load(path_det2)

In [ ]:
DATASETS_LIST = [
    "run_3_revisited_5may2021",
    "Sodexo_revisited_8jun",
    "Test_Empoli_01_06_2021",
    "Test_Empoli_08_06_2021",
    "Test_Empoli_09_06_2021",
    "Test_Empoli_26_05_2021",
    "Test_Empoli_27_05_2021",
    "Test_Empoli_28_05_2021",
    "Test_Empoli_29_04_2021",
    "Test_Empoli_30_04_2021",
    "Test_Empoli_31_05_2021",
    "Test_Empoli_from_10_06_2021_to_30_06_2021",
    "UNIMIB_revisited_7giu2021"
]

BASE_PATH = Path('/mnt/data/smarttray/dataset_05jul2021_av3b')

In [ ]:
def initialize_pascalvoc_header(img_path, W, H):
    node_root = Element('annotation')

    node_folder = SubElement(node_root, 'folder')
    node_folder.text = f'{BASE_PATH/img_path}'

    node_filename = SubElement(node_root, 'filename')
    node_filename.text = f'{img_path.name}'

    node_size = SubElement(node_root, 'size')
    node_width = SubElement(node_size, 'width')
    node_width.text = f'{W}'

    node_height = SubElement(node_size, 'height')
    node_height.text = f'{H}'

    node_depth = SubElement(node_size, 'depth')
    node_depth.text = '3'
    
    return node_root

def add_bb_to_xml(node_root, bb, scr, class_name, food_scr=None):
    node_object = SubElement(node_root, 'object')
    
    node_name = SubElement(node_object, 'name')
    node_name.text = f'{class_name}'
    node_difficult = SubElement(node_object, 'difficult')
    node_difficult.text = '0'
    
    node_bndbox = SubElement(node_object, 'bndbox')
    
    node_xmin = SubElement(node_bndbox, 'xmin')
    node_xmin.text = f'{bb[0]}'
    node_ymin = SubElement(node_bndbox, 'ymin')
    node_ymin.text = f'{bb[1]}'
    node_xmax = SubElement(node_bndbox, 'xmax')
    node_xmax.text = f'{bb[2]}'
    node_ymax = SubElement(node_bndbox, 'ymax')
    node_ymax.text = f'{bb[3]}'
    
    node_score = SubElement(node_bndbox, 'score')
    node_score.text = f'{scr}'
    
    if food_scr: 
        node_food_scr = SubElement(node_bndbox, 'food_score')
        node_food_scr.text = f'{food_scr}'

    return node_root

In [ ]:
def debug(img, food_bbs, rcp_bbs):            
    draw_bbs(img, food_bbs, color=blue)
    draw_bbs(img, rcp_bbs, color=red)

    plot(img); plt.show()
    x = input("write 'c' to stop")
    clear_output()
    if x=='c': raise Exception() 

In [ ]:
def get_n_best_bbs(bbs, scrs, n):
    x = list(zip(bbs,scrs))
    x = sorted(x, key=lambda x: x[1], reverse=True)[:n]
    bbs, scrs = list(zip(*x))
    return bbs, scrs

In [ ]:
bbs, scrs = [[1], [3], [2], [5]], [1, 3, 2, 5]
test_eq(get_n_best_bbs(bbs, scrs, 2), ([[5], [3]], [5, 3]))

In [ ]:
def save_xml(node_root, xml_path):
    xml_path = Path(xml_path)
    xml_path.parent.mkdir(exist_ok=True)

    xml = tostring(node_root) #, pretty_print=True Formatted display, the newline of the newline
    dom = parseString(xml)

    with open(str(xml_path), 'w') as f: 
        dom.writexml(f,indent='   ', addindent='   ', newl='\n')
        
def create_xml_anns_of_img(img_path, do_debug, det1_th, det2_th, n_best):            
    img = load_image(img_path).numpy()
    H, W, _ = img.shape        
    node_root = initialize_pascalvoc_header(img_path, W, H)

    food_bbs, food_scrs = get_bbs(img, det1, threshold=det1_th, n_best=n_best, in_absolute_coords=True)
    for bb, scr  in zip(food_bbs, food_scrs): 
        node_root = add_bb_to_xml(node_root, bb, scr, "food")

    foods = get_bb_crops(img, food_bbs)
    for food, food_bb, food_scr in zip(foods, food_bbs, food_scrs):
        X, Y, w, h = food_bb.coco_coords
        if w > 700 and h > 700:
            rcp_bbs, rcp_scrs = get_bbs(food, det2, threshold=det2_th, n_best=n_best, in_absolute_coords=True)
            for bb, scr  in zip(rcp_bbs, rcp_scrs): 
                bb.shift(X, Y)
                node_root = add_bb_to_xml(node_root, bb, scr, "recipe", food_scr)                

            if do_debug: debug(img, food_bbs, rcp_bbs)

    return node_root


In [ ]:
det1_th = 0.0
det2_th = 0.0
n_best = 20

blue = [0,0,255]
red  = [255,0,0]

do_debug = False

# for ds in tqdm(DATASETS_LIST):
#     ds_path = BASE_PATH/ds/'val/images'
#     if ds_path.exists(): img_paths = [img_path for img_path in ds_path.iterdir() if not img_path.is_dir()]
#     else: continue

#     for img_path in tqdm(img_paths):
#         node_root = create_xml_anns_of_img(img_path, do_debug, det1_th, det2_th, n_best)
#         xml_path = BASE_PATH/'two_detector_results'/ds/f'{img_path.stem}.xml'
#         save_xml(node_root, xml_path)

data_test_path = BASE_PATH/'data_test/images'
img_paths = [img_path for img_path in data_test_path.iterdir() if not img_path.is_dir()]
for img_path in tqdm(img_paths):
    node_root = create_xml_anns_of_img(img_path, do_debug, det1_th, det2_th, n_best)
    xml_path = BASE_PATH/'two_detector_results'/f'data_test/{img_path.stem}.xml'
    save_xml(node_root, xml_path)    

  0%|          | 0/1504 [00:00<?, ?it/s]

In [ ]:
read_xml_anns(xml_path)

NameError: name 'xml_path' is not defined